In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.decomposition import PCA


In [2]:
def split_scalar(X,y):
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    return X_train,X_test,y_train,y_test


In [3]:
def perform_pca(X_train,X_test,n):
    pca = PCA(n_components=n)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)
    explained_variance = pca.explained_variance_ratio_
    return pca,X_train_pca, X_test_pca,explained_variance


In [4]:
def logistic(X_train_pca,y_train,X_test_pca):
    from sklearn.linear_model import LogisticRegression
    classifier = LogisticRegression(random_state = 0)
    classifier.fit(X_train_pca, y_train)
    cm,accuracy,classifier = cm_predition(classifier,X_test_pca)
    return cm,accuracy,classifier

def svm_linear(X_train_pca,y_train,X_test_pca):
    from sklearn.svm import SVC
    classifier = SVC(kernel = 'linear', random_state = 0)
    classifier.fit(X_train_pca, y_train)
    cm,accuracy,classifier = cm_predition(classifier,X_test_pca)
    return  cm,accuracy,classifier
           
def svm_NL(X_train_pca,y_train,X_test_pca):
    from sklearn.svm import SVC
    classifier = SVC(kernel = 'rbf', random_state = 0)
    classifier.fit(X_train_pca, y_train)
    cm,accuracy,classifier = cm_predition(classifier,X_test_pca)
    return  cm,accuracy,classifier

def Navie(X_train_pca,y_train,X_test_pca):       
    # Fitting K-NN to the Training set
    from sklearn.naive_bayes import GaussianNB
    classifier = GaussianNB()
    classifier.fit(X_train_pca, y_train)
    cm,accuracy,classifier = cm_predition(classifier,X_test_pca)
    return  cm,accuracy,classifier 

def knn(X_train_pca,y_train,X_test_pca):
    # Fitting K-NN to the Training set
    from sklearn.neighbors import KNeighborsClassifier
    classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
    classifier.fit(X_train_pca, y_train)
    cm,accuracy,classifier = cm_predition(classifier,X_test_pca)
    return  cm,accuracy,classifier

def Decision(X_train_pca,y_train,X_test_pca):
    # Fitting K-NN to the Training set
    from sklearn.tree import DecisionTreeClassifier
    classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
    classifier.fit(X_train_pca, y_train)
    cm,accuracy,classifier = cm_predition(classifier,X_test_pca)
    return  cm,accuracy,classifier     

def random(X_train_pca,y_train,X_test_pca):
    # Fitting K-NN to the Training set
    from sklearn.ensemble import RandomForestClassifier
    classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
    classifier.fit(X_train_pca, y_train)
    cm,accuracy,classifier = cm_predition(classifier,X_test_pca)
    return  cm,accuracy,classifier


In [5]:
def cm_predition (classifier,X_test_pca):
    y_pred = classifier.predict(X_test_pca)
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y_test, y_pred)
    from sklearn.metrics import accuracy_score
    accuracy= accuracy_score(y_test, y_pred)
    return cm,accuracy,classifier

In [6]:
def pca_merge(acclog,accsvmlin,accsvmnl,accnav,accknn,accdec,accrand): 
    dataframe=pd.DataFrame(index=['PCA'],columns=['Logistic','SVMlin','SVMNL','Navie','KNN','DT','RF'])
    for number,index in enumerate(dataframe.index):      
        dataframe['Logistic'][index]=acclog[number]
        dataframe['SVMlin'][index]=accsvmlin[number] 
        dataframe['SVMNL'][index]=accsvmnl[number] 
        dataframe['Navie'][index]=accnav[number] 
        dataframe['KNN'][index]=accknn[number] 
        dataframe['DT'][index]=accdec[number] 
        dataframe['RF'][index]=accrand[number] 
    return dataframe


In [7]:
dataset1=pd.read_csv("prep.csv",index_col=None)

df2=dataset1

df2 = pd.get_dummies(df2, drop_first=True)

X=df2.drop('classification_yes', 1)
y=df2['classification_yes']

In [8]:
# created a empty list to append the final value
acclog=[]
accsvmlin=[]
accsvmnl=[]
accnav=[]
accknn=[]
accdec=[]
accrand=[]

X_train, X_test, y_train, y_test=split_scalar(X,y) 

pca,X_train_pca,X_test_pca,explained_variance = perform_pca(X_train,X_test,4)

cm,accuracy,classifier=logistic(X_train_pca,y_train,X_test_pca)
acclog.append(accuracy)

cm,accuracy,classifier=svm_linear(X_train_pca,y_train,X_test_pca)
accsvmlin.append(accuracy)

cm,accuracy,classifier=svm_NL(X_train_pca,y_train,X_test_pca)
accsvmnl.append(accuracy)

cm,accuracy,classifier=Navie(X_train_pca,y_train,X_test_pca)
accnav.append(accuracy)

cm,accuracy,classifier=knn(X_train_pca,y_train,X_test_pca)
accknn.append(accuracy)

cm,accuracy,classifier=Decision(X_train_pca,y_train,X_test_pca)
accdec.append(accuracy)

cm,accuracy,classifier=random(X_train_pca,y_train,X_test_pca)
accrand.append(accuracy)

result = pca_merge(acclog,accsvmlin,accsvmnl,accnav,accknn,accdec,accrand)

In [9]:
result

,Logistic,SVMlin,SVMNL,Navie,KNN,DT,RF
PCA,0.9875,0.9875,1,0.9875,1,1,1
